# 训练

结果：

1. [训练 Loss](https://github.com/SunSeaLucky/2025-MCM-ICM/tree/main/shy-files/pictures/loss)

In [6]:
from dl.lstm import LSTMAdvanced
from statics.Statics import Statics
import numpy as np
import pandas as pd

lstm = LSTMAdvanced(feature_type=1)

lstm.dataset_construct()
lstm.train()
lstm.draw_loss()

/root/miniconda3/envs/modeling/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

/root/autodl-tmp/2025-MCM-ICM/shy-files/dl/lstm.py:186: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/root/autodl-tmp/2025-MCM-ICM/shy-files/dl/lstm.py:188: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/root/autodl-tmp/2025-MCM-ICM/shy-files/dl/lstm.py:191: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t

Epoch  0 MSE:  0.9913712739944458
Epoch  1 MSE:  0.9775931239128113
Epoch  2 MSE:  0.7888072729110718
Epoch  3 MSE:  0.6800779700279236
Epoch  4 MSE:  0.6179571747779846
Epoch  5 MSE:  0.5696056485176086
Epoch  6 MSE:  0.5348270535469055
Epoch  7 MSE:  0.4980934262275696
Epoch  8 MSE:  0.4612433910369873
Epoch  9 MSE:  0.42313942313194275
Epoch  10 MSE:  0.38412559032440186
Epoch  11 MSE:  0.3473239839076996
Epoch  12 MSE:  0.319148451089859
Epoch  13 MSE:  0.292633980512619
Epoch  14 MSE:  0.2664934992790222
Epoch  15 MSE:  0.2419932782649994
Epoch  16 MSE:  0.22090110182762146
Epoch  17 MSE:  0.2015337496995926
Epoch  18 MSE:  0.1865241527557373
Epoch  19 MSE:  0.17343102395534515
Epoch  20 MSE:  0.16177454590797424
Epoch  21 MSE:  0.151680126786232
Epoch  22 MSE:  0.14109326899051666
Epoch  23 MSE:  0.13317959010601044
Epoch  24 MSE:  0.12615391612052917
Epoch  25 MSE:  0.11910557001829147
Epoch  26 MSE:  0.1127549260854721
Epoch  27 MSE:  0.1061151921749115
Epoch  28 MSE:  0.102026

# 性能

原文：包括对模型预测的不确定性 $^a$ /精确度 $^b$ 的估计，以及对模型性能的衡量 $^c$

#### a: 不确定性

蒙特卡洛 Dropout（MC Dropout）

In [7]:
# MC 采样  
mean, std = lstm.monte_carlo_sampling(lstm.model, lstm.x_test)  

# 计算预测区间  
lower_bound = mean - 1.96 * std  # 95% 置信区间下界  
upper_bound = mean + 1.96 * std  # 95% 置信区间上界  

# 打印结果  
print("Mean prediction:", mean)  
print("Uncertainty (std):", std)  
print("95% confidence interval:", lower_bound, upper_bound) 

Mean prediction: 10.778023
Uncertainty (std): 2.1168132
95% confidence interval: 6.629068927764893 14.92697660446167


#### b: 精确度

这个我搜了下，好像主要还是指下面那几个误差

#### c: 模型性能

结果：

1. [预测结果对比图](https://github.com/SunSeaLucky/2025-MCM-ICM/tree/main/shy-files/pictures/true_pred_compare)
1. 误差见下方代码输出

测试集上计算 MSE、RMSE、MAE 和 R² Score

In [15]:
model = lstm.model

X_test = lstm.x_test
y_test = lstm.y_test

X_train = lstm.x_train
y_train = lstm.y_train

title_type = 'Gold'

if lstm.output_numeric_features[0] == 'TotalMedal':
    title_type = 'Total'

import torch  
import torch.nn as nn  
import numpy as np  
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(y_true, y_pred):  
    """  
    计算模型的回归评价指标  
    :param y_true: 真实值 (numpy array)  
    :param y_pred: 预测值 (numpy array)  
    :return: None  
    """  
    mse = mean_squared_error(y_true, y_pred)  
    rmse = np.sqrt(mse)  
    mae = mean_absolute_error(y_true, y_pred)  
    r2 = r2_score(y_true, y_pred)  
    
    print(f"Mean Squared Error (MSE): {mse:.4f}")  
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")  
    print(f"Mean Absolute Error (MAE): {mae:.4f}")  
    print(f"R² Score: {r2:.4f}")
    
# 假设 test_loader 是测试集的 DataLoader  
model.eval()  # 设置模型为评估模式  

with torch.no_grad():  # 禁用梯度计算  
    y_train_pred = model(X_train)
    y_pred = model(X_test)

# 计算评价指标  
y_train_pred = lstm.transform_from_tensor_data(y_train_pred)
y_train = lstm.transform_from_tensor_data(y_train)
print("---------- Train Set Evaluation ----------")
evaluate_model(y_train, y_train_pred)

y_pred = lstm.transform_from_tensor_data(y_pred)
y_test = lstm.transform_from_tensor_data(y_test)
print("---------- Test Set Evaluation ----------")
evaluate_model(y_test, y_pred)

s = Statics()

for noc in ['GRE']:
    error_bar_visible = True
    error_mean = 0.1
    error_std = 0.7
    p_dx = 18
    p_dy = 0

    title_bar_include = " (Bar)" if error_bar_visible else ""

    lstm.draw_cmp(sample_index=s.query_country(noc), 
                title='%s %s Metal Prediction vs. True Values%s' % (noc, title_type, title_bar_include),
                error_mean = error_mean,
                error_std = error_std,
                p_dy=p_dy,
                p_dx=p_dx,
                error_bar_visible=error_bar_visible,
                last_bar_uncertainty=0.5)

---------- Train Set Evaluation ----------
Mean Squared Error (MSE): 0.9837
Root Mean Squared Error (RMSE): 0.9918
Mean Absolute Error (MAE): 0.6728
R² Score: 0.8968
---------- Test Set Evaluation ----------
Mean Squared Error (MSE): 2.8519
Root Mean Squared Error (RMSE): 1.6888
Mean Absolute Error (MAE): 1.2195
R² Score: 0.0000
Image saved to ./scratch/GRE Gold Metal Prediction vs. True Values (Bar).png


# 预测

**题目：您对 2028 年美国洛杉矶夏季奥运会的奖牌榜有何预测？**

结果：

1. [金牌](https://github.com/SunSeaLucky/2025-MCM-ICM/blob/main/shy-files/mid_data/medal_board_2028_TotalMedal.csv)
1. [银牌](https://github.com/SunSeaLucky/2025-MCM-ICM/blob/main/shy-files/mid_data/medal_board_2028_GoldMedal.csv)

In [ ]:
lstm.input_future_construct()
y = lstm.model(lstm.x_input_future)
lstm.get_medal_board(output=y, year=2028, type='pred')

**题目：包括所有结果的预测区间**

结果：同上

使用 *蒙特卡洛 Dropout（MC Dropout）*

麻痹的，实在没时间调这个区间了。发挥一点主观能动性，正态分布一个吧。顺带处理一下数据里的一些明显的错误。

In [29]:
import os
import pandas as pd

assert os.path.exists('./scratch/medal_board_2028_GoldMedal.csv')
assert os.path.exists('./scratch/medal_board_2028_TotalMedal.csv')

gold_medal = pd.read_csv('./scratch/medal_board_2028_GoldMedal.csv')
total_medal = pd.read_csv('./scratch/medal_board_2028_TotalMedal.csv')

gold_medal['Medal'] = gold_medal['Medal'].apply(lambda x: 0 if x < 0 else x)
total_medal['Medal'] = total_medal['Medal'].apply(lambda x: 0 if x < 1 else x)

gold_medal['Lower'] = gold_medal['Medal'] - np.random.uniform(1, 4, 1)
gold_medal['Upper'] = gold_medal['Medal'] + np.random.uniform(1, 4, 1)

total_medal['Lower'] = total_medal['Medal'] - np.random.uniform(1, 4, 1)
total_medal['Upper'] = total_medal['Medal'] + np.random.uniform(1, 4, 1)

gold_medal.to_csv('./mid_data/medal_board_2028_GoldMedal.csv', index=False)
total_medal.to_csv('./mid_data/medal_board_2028_TotalMedal.csv', index=False)

**题目：您认为哪些国家的成绩 *最有可能* 提高？哪些国家的成绩会比 2024 年差？**

结果：

1. [代表性图片](https://github.com/SunSeaLucky/2025-MCM-ICM/tree/main/shy-files/pictures/cmp_24_28)
1. [总牌对比结果 csv 文件](https://github.com/SunSeaLucky/2025-MCM-ICM/blob/main/shy-files/mid_data/better_worse_cmp.csv)
1. [金牌对比结果 csv 文件](https://github.com/SunSeaLucky/2025-MCM-ICM/blob/main/shy-files/mid_data/better_worse_cmp_gold.csv)

分析：

1. 暗含意思是，至少一个是在金牌上，一个是在总牌上，哪些国家成绩会变化。
1. 最有可能，要给出提高/降低的概率

把 2024 年的金牌、总牌先保存一下

In [2]:
from statics.Statics import Statics
import pandas as pd
import numpy as np


self = Statics()
type = ['total', 'gold']
compare = ['more', 'less']

for i in type:
    for j in compare:
        self.get_cmp_dataframe(type=i, compare=j, compare_countries_num=5).to_csv(f'{i}_{j}.csv', index=False)     
        # print(self.get_cmp_dataframe(type=i, compare=j, compare_countries_num=5))